#### Importing necessary libraries

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # This avoids SettingWithCopyWarnings
import numpy as np

In [4]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

#### Initializing Variables


In [3]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data


### Heiken Ashi is one type of chart pattern from Japan that filters out some noise in an effort to better capture the trend. 

In [5]:
def HA(df):
    df['HA_Close']=(df['Open']+ df['High']+ df['Low']+ df['Close'])/4
    df['HA_Open']=(df['Open']+df['Close'])/2   
    
    for i in range(1, len(df)):
        df['HA_Open'][i]=(df['HA_Open'][i-1]+df['HA_Close'][i-1])/2 
    df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)
    return 

### Set of Stocks

In [6]:
StockList=['INFY', 'ICICIBANK'] 

### Fetching data using Yahoo API,  Setting up Pandas Dataframe in the dictionary 'data' with key as Stock name and Value as DataFrame


In [7]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

[*********************100%***********************]  1 of 1 downloaded

#### Sometimes due to rate limiting factor, Yahoo api when called yeilds no data. Below cell helps us out in figuring whether we need to ping the api again. 

In [8]:
for stock in data:
    if data[stock].empty:
        print stock

In [9]:
for stock in data:
    print stock, data[stock].head()

ICICIBANK                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-01  237.544998  239.636002  234.544998  239.091003  229.999863   
2016-01-04  237.272995  237.591003  231.500000  232.317993  223.484390   
2016-01-05  232.955002  234.091003  228.817993  233.363998  224.490616   
2016-01-06  232.182007  233.408997  226.544998  227.363998  218.718781   
2016-01-07  224.000000  225.182007  221.000000  224.317993  215.788589   

              Volume  
Date                  
2016-01-01   5998097  
2016-01-04   9435792  
2016-01-05   8966978  
2016-01-06  17416181  
2016-01-07  18240713  
INFY                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2016-01-01  1100.000000  1108.949951  1093.199951  1105.250000  1049.958252   
2016-01-04  1099.949951  1102.449951  1076.050049  1078.900024  1024

###  Data Cleaning, Column renaming, etc

In [10]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    data[stock]=data[stock][['Open','High','Low','Close']] # Removing unwanted columns

### Creating  Heiken Ashi candles 

In [11]:
for stock in data:    
    HA(data[stock])
    data[stock]=data[stock][['HA_Open','HA_High','HA_Low','HA_Close']] # Removing unwanted columns
    data[stock]=data[stock].round(2)

In [12]:
data['INFY'].head()

,HA_Open,HA_High,HA_Low,HA_Close
Date,,,,
2016-01-01,1102.62,1108.95,1093.20,1101.85
2016-01-04,1102.24,1102.45,1076.05,1089.34
2016-01-05,1095.79,1095.79,1062.25,1076.83
2016-01-06,1086.31,1086.31,1058.90,1069.39
2016-01-07,1077.85,1077.85,1047.90,1056.66


In [13]:
data['ICICIBANK'].head(10)

,HA_Open,HA_High,HA_Low,HA_Close
Date,,,,
2016-01-01,238.32,239.64,234.54,237.70
2016-01-04,238.01,238.01,231.50,234.67
2016-01-05,236.34,236.34,228.82,232.31
2016-01-06,234.32,234.32,226.54,229.88
2016-01-07,232.10,232.10,221.00,223.62
2016-01-08,227.86,227.86,221.32,222.91
2016-01-11,225.39,225.39,216.41,218.93
2016-01-12,222.16,222.16,214.18,217.09
2016-01-13,219.62,220.82,210.09,216.20


### Taking data to excel

In [14]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()